In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Aljazeera web scrapping

In this data scraping process, the goal was to extract information from the Al Jazeera website. The first step involved scraping the "Category" section to gather the URLs corresponding to different categories. These URLs were then used to navigate to each category's page and extract the URLs of individual articles along with their titles. After obtaining the article URLs and titles, the next step was to extract the text content from each article. Finally, all the scraped data was consolidated into a pandas DataFrame with six columns: "id" (unique identifier), "url" (article URL), "titre" (article title), "categorie" (category), "ArabCategory" (optional Arabic category), and "texte" (article text). The DataFrame consists of 350 entries and provides a comprehensive collection of data for further analysis or processing.

In [2]:
import requests
from bs4 import BeautifulSoup
import csv

# Make a GET request to the URL of the page you want to scrape
url = "https://www.aljazeera.com/"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
# Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
# Find the ul tag containing the items
ul_element = soup.find("ul", class_="menu header-menu")

categories = []
links = []

# Iterate over the items in the ul tag to extract the categories and links
for li_element in ul_element.find_all("li", class_="menu__item"):
    category = li_element.find("span").text.strip()
    link = li_element.find("a")["href"]
    categories.append(category)
    links.append(link)

# Create a list of dictionaries
data = []
for category, link in zip(categories, links):
    data.append({"Category": category, "Link": link})

# Write the data to a CSV file
with open("categories.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["Category", "Link"])
    writer.writeheader()  # Write the CSV header
    writer.writerows(data)

print("Extraction completed. The data has been saved in 'categories.csv'.")

Extraction completed. The data has been saved in 'categories.csv'.


In [4]:
# Load the data/kaggle/working/categories2.csv
df = pd.read_csv("/kaggle/working/categories.csv", nrows=100)
df.head()

,Category,Link
0,News,/news/
1,Middle East,/middle-east/
2,Africa,/africa/
3,Asia,/asia/
4,US & Canada,/us-canada/


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  23 non-null     object
 1   Link      23 non-null     object
dtypes: object(2)
memory usage: 496.0+ bytes


In [9]:
import requests
from bs4 import BeautifulSoup
import csv

# Make a GET request to the URL of the page you want to scrape
url = "https://www.aljazeera.com//"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the ul tag containing the links
    ul_element = soup.find("ul", class_="menu header-menu")

    # Extract the URLs
    urls = []
    for li_element in ul_element.find_all("li", class_="menu__item"):
        link = "https://www.aljazeera.com/" + li_element.find("a")["href"]
        urls.append(link)

# Créer une liste pour stocker les informations des articles
articles_data = []
article_id = 0

# Parcourir chaque URL
for url in urls:
    # Envoyer une requête GET à l'URL
    response = requests.get(url)

    # Vérifier que la requête s'est bien déroulée
    if response.status_code == 200:
        # Utiliser Beautiful Soup pour analyser le contenu HTML de la page
        soup = BeautifulSoup(response.content, "html.parser")

        # Trouver tous les éléments <a> contenant les liens des articles
        article_links = soup.find_all("a", class_="u-clickable-card__link")

        # Extraire les informations des articles
        for link in article_links:
            article_id += 1
            article_url = "https://www.aljazeera.com/" + link["href"]
            article_title = link.span.get_text()

            # Extraire la catégorie de l'URL
            category = url.split("/")[-2]

            # Ajouter les informations dans la liste
            articles_data.append({"id": article_id, "url": article_url, "titre": article_title, "categorie": category})

# Écrire les données dans un fichier CSV
with open("articles.csv", "w", encoding="utf-8", newline="") as csvfile:
    fieldnames = ["id", "url", "titre", "categorie"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Écrire les en-têtes du fichier CSV
    writer.writeheader()

    # Écrire les données des articles
    writer.writerows(articles_data)

print("Le fichier CSV a été généré avec succès.")

Le fichier CSV a été généré avec succès.


In [10]:
# Load the data
df = pd.read_csv("/kaggle/working/articles.csv")
df.head()


,id,url,titre,categorie
0,1,https://www.aljazeera.com//news/liveblog/2023/...,‘Un­law­ful’: UN aid chief de­cries Is­raeli a...,news
1,2,https://www.aljazeera.com//gallery/2023/12/29/...,Pho­tos: Pales­tini­ans per­form Fri­day praye...,news
2,3,https://www.aljazeera.com//news/2023/12/29/arg...,Ar­genti­na an­nounces that it will not join B...,news
3,4,https://www.aljazeera.com//program/newsfeed/20...,Rus­sia launch­es “most mas­sive aer­i­al at­t...,news
4,5,https://www.aljazeera.com//news/2023/12/29/sou...,South Africa files case at ICJ ac­cus­ing Is­r...,news


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         256 non-null    int64 
 1   url        256 non-null    object
 2   titre      256 non-null    object
 3   categorie  256 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.1+ KB


In [14]:
import requests
from bs4 import BeautifulSoup
import csv

# Make a GET request to the URL of the page you want to scrape
url = "https://www.aljazeera.com/"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the ul tag containing the links
    ul_element = soup.find("ul", class_="menu header-menu")

    # Extract the URLs
    urls = []
    for li_element in ul_element.find_all("li", class_="menu__item"):
        link = "https://www.aljazeera.com/" + li_element.find("a")["href"]
        urls.append(link)

    # Create a list to store the article information
    articles_data = []
    article_id = 0

    # Iterate over each URL
    for url in urls:
        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Use Beautiful Soup to parse the HTML content of the page
            soup = BeautifulSoup(response.content, "html.parser")

            # Find all the <a> elements containing the article links
            article_links = soup.find_all("a", class_="u-clickable-card__link")

            # Extract the article information
            for link in article_links:
                article_id += 1
                article_url = "https://www.aljazeera.com/" + link["href"]
                article_title = link.span.get_text()

                # Extract the article summary from the HTML content
                summary = link.find_next("div", class_="gc__excerpt").p.get_text()

                # Extract the category from the URL
                category = url.split("/")[-2]

                # Add the information to the list
                articles_data.append({"id": article_id, "url": article_url, "title": article_title, "category": category, "summary": summary})

    # Write the data to a CSV file
    with open("articles.csv", "w", encoding="utf-8", newline="") as csvfile:
        fieldnames = ["id", "url", "title", "category", "summary"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the CSV file headers
        writer.writeheader()

        # Write the article data
        writer.writerows(articles_data)

    print("The CSV file has been generated successfully.")

The CSV file has been generated successfully.


In [15]:
# Load the data
df = pd.read_csv("/kaggle/working/articles.csv")
df.head()

,id,url,title,category,summary
0,1,https://www.aljazeera.com//news/liveblog/2023/...,‘Un­law­ful’: UN aid chief de­cries Is­raeli a...,news,UN­R­WA says Is­raeli forces fired on aid con­...
1,2,https://www.aljazeera.com//gallery/2023/12/29/...,Pho­tos: Pales­tini­ans per­form Fri­day praye...,news,Is­rael re­stricts Pales­tini­ans’ ac­cess to ...
2,3,https://www.aljazeera.com//news/2023/12/29/arg...,Ar­genti­na an­nounces that it will not join B...,news,The move is the lat­est shift in eco­nom­ic an...
3,4,https://www.aljazeera.com//program/newsfeed/20...,Rus­sia launch­es “most mas­sive aer­i­al at­t...,news,Rus­sia launch­es one of its largest nights of...
4,5,https://www.aljazeera.com//news/2023/12/29/sou...,South Africa files case at ICJ ac­cus­ing Is­r...,news,"Is­rael, which has been ac­cused of met­ing ou..."


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        256 non-null    int64 
 1   url       256 non-null    object
 2   title     256 non-null    object
 3   category  256 non-null    object
 4   summary   256 non-null    object
dtypes: int64(1), object(4)
memory usage: 10.1+ KB


In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Charger le DataFrame contenant les URLs
df = pd.read_csv("/kaggle/working/articles.csv")

# Parcourir chaque ligne du DataFrame
for index, row in df.iterrows():
    url = row['url']
    response = requests.get(url)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Analyser le contenu HTML avec BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Extraire le texte des balises <p>
        paragraphs = soup.find_all("p")

        # Concaténer le texte extrait des balises <p>
        text = ' '.join(p.get_text() for p in paragraphs)

        # Mettre à jour la colonne "texte" du DataFrame avec le texte extrait
        df.at[index, 'texte'] = text

    else:
        print(f"La requête a échoué pour l'URL {url}")

# Enregistrer le DataFrame mis à jour avec le texte extrait
df.to_csv('Aljazeera_dataset_eng.csv', index=False)

In [23]:


# Enregistrer le DataFrame mis à jour avec le texte extrait
df.to_csv('Aljazeera_dataset_eng.csv', index=False)

In [18]:

# Charger le DataFrame contenant les URLs
df = pd.read_csv("/kaggle/working/Aljazeera_dataset.csv")
df.head(50)

,id,url,title,category,summary,texte
0,1,https://www.aljazeera.com//news/liveblog/2023/...,‘Un­law­ful’: UN aid chief de­cries Is­raeli a...,news,UN­R­WA says Is­raeli forces fired on aid con­...,UNRWA says Israeli forces fired on aid convoy ...
1,2,https://www.aljazeera.com//gallery/2023/12/29/...,Pho­tos: Pales­tini­ans per­form Fri­day praye...,news,Is­rael re­stricts Pales­tini­ans’ ac­cess to ...,In Pictures Israeli authorities barred Palesti...
2,3,https://www.aljazeera.com//news/2023/12/29/arg...,Ar­genti­na an­nounces that it will not join B...,news,The move is the lat­est shift in eco­nom­ic an...,The move is the latest shift in economic and f...
3,4,https://www.aljazeera.com//program/newsfeed/20...,Rus­sia launch­es “most mas­sive aer­i­al at­t...,news,Rus­sia launch­es one of its largest nights of...,An overnight barrage of Russian missiles and d...
4,5,https://www.aljazeera.com//news/2023/12/29/sou...,South Africa files case at ICJ ac­cus­ing Is­r...,news,"Is­rael, which has been ac­cused of met­ing ou...","Israel, which has been accused of meting out c..."
5,6,https://www.aljazeera.com//news/2023/12/29/pol...,Poland says Russ­ian mis­sile briefly en­tered...,news,NATO chief Jens Stoltenberg says or­gan­i­sa­t...,NATO chief Jens Stoltenberg says organisation ...
6,7,https://www.aljazeera.com//news/2023/12/29/ten...,‘Forced dis­place­ment’: Pales­tin­ian en­voy ...,news,Is­rael in­ten­si­fies air and ground of­fen­s...,Israel has escalated its unprecedented air and...
7,8,https://www.aljazeera.com//news/2023/12/29/us-...,The lat­est tool for pre­vent­ing over­dos­es?...,news,Vend­ing ma­chines with free over­dose-re­ver­...,Vending machines with free overdose-reversal d...
8,9,https://www.aljazeera.com//news/2023/12/29/ind...,In­dia for­mal­ly asks Pak­istan to ex­tra­dit...,news,Saeed is ac­cused by In­dia and the US of be­i...,Saeed is accused by India and the US of being ...
9,10,https://www.aljazeera.com//news/2023/12/29/chi...,Chi­na ap­points for­mer navy chief Dong Jun a...,news,"Dong re­places Li Shang­fu, who was fired and ...","Dong replaces Li Shangfu, who was fired and di..."


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        256 non-null    int64 
 1   url       256 non-null    object
 2   title     256 non-null    object
 3   category  256 non-null    object
 4   summary   256 non-null    object
 5   texte     256 non-null    object
dtypes: int64(1), object(5)
memory usage: 12.1+ KB


In [ ]:
text1 = df.iloc[0][-1]
print(text1)

In [21]:
summ1 = df.iloc[0][-2]
print(summ1)

UN­R­WA says Is­raeli forces fired on aid con­voy trav­el­ling on des­ig­nat­ed route; at­tacks on civil­ians con­tin­ue across Gaza.


/tmp/ipykernel_42/1705048274.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  summ1 = df.iloc[0][-2]
